In [10]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.base import ClassifierMixin
from classifierEnsemble import ClassifierEnsemble

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

In [11]:
testdf = pd.read_csv("data/water_quality_test.csv")
traindf = pd.read_csv("data/water_quality_train.csv")
testdf['Potability'] = testdf['Potability'].apply(lambda x: 1 if x == "Yes" else 0)
traindf['Potability'] = traindf['Potability'].apply(lambda x: 1 if x == "Yes" else 0 if x == "No" else x)

traindf
# traindfnotna = traindf.dropna()



,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,169.974849,23403.637304,8.519730,NaN,475.573562,12.924107,50.861913,2.747313,1.0
1,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,NaN
2,5.934279,223.858125,23249.653834,4.602850,NaN,277.384537,11.366863,66.623944,5.217895,1.0
3,6.193170,225.683422,12883.726496,4.739360,NaN,389.405978,14.560136,76.457816,3.466645,NaN
4,NaN,204.852187,13286.156575,9.093843,332.845544,365.687520,8.589896,76.306376,4.268525,NaN
...,...,...,...,...,...,...,...,...,...,...
2759,8.347949,231.203101,13533.230033,7.623604,313.671127,527.380871,18.663901,58.052866,4.217861,NaN
2760,6.836225,166.742962,24293.172663,6.861235,332.881582,390.804955,16.006991,87.101925,3.666614,NaN
2761,6.385471,231.488458,26756.918000,8.774357,NaN,422.067701,16.563116,73.260640,4.681717,NaN
2762,6.490037,205.541517,19397.323889,7.894245,368.561073,479.786195,15.805085,56.982020,2.430883,NaN


In [12]:
xTrain = traindf.drop('Potability', axis=1)
yTrain = pd.DataFrame(traindf['Potability'])
xTest = testdf.drop('Potability', axis=1)
yTest = pd.DataFrame(testdf['Potability'])

In [13]:
# Lista de clasificadores
classifierList: list[ClassifierMixin] = [
    RandomForestClassifier(),
    MLPClassifier(), 
    LogisticRegression()
]

classifierWeights = [
    1,
    1,
    1
]

In [14]:
# Convertimos los clasificadores en semi-supervisados

classifierList: list[tuple[str, ClassifierMixin]] = [(str(c), SelfTrainingClassifier(c)) for c in classifierList]

In [15]:
classEn = ClassifierEnsemble(classifierList)

In [16]:
classEn.fit(xTrain, yTrain)

[             ph    Hardness        Solids  Chloramines     Sulfate   
18     7.581688  180.749140  11989.246243     4.977307  328.176978  \
687         NaN         NaN           NaN          NaN         NaN   
136    8.783597  199.930278  19471.465438     7.415406  319.737009   
892   10.391942  262.741770  39116.682706     3.205786  285.728420   
1030   7.490808  198.966512   6873.559885     8.534329         NaN   
...         ...         ...           ...          ...         ...   
512    7.566517  205.396582  30823.730490     7.816636  354.175972   
873    4.808040  186.160570  19118.050299     7.054444  359.406236   
1647   7.074251  208.920369  16739.266635     7.861469         NaN   
2251   9.514627  215.725527  16553.562437     7.547859  321.776213   
65          NaN         NaN           NaN          NaN         NaN   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  Potability  
18      617.883513       13.561253        39.215917   4.457282         NaN  
687 

In [17]:
# Creamos el ensemble por votación
# votingEnsemble = VotingClassifier(classifierList, weights=classifierWeights)

In [18]:
# votingEnsemble.fit(xTrain, yTrain)
# votingEnsemble.get_params()
# # votingEnsemble.score(xTest, yTest)